# PSyclone reverse mode automatic differentiation
## `psyclone.autodiff`

### 1. Differentiating some outputs of a subroutine with respect to some input variables

Let us first define two trivial Fortran subroutines.

In [2]:

source = """
subroutine qux(a, b, w)
    implicit none
    real, intent(inout) :: a(10, 10)
    real, intent(out) :: b(10, 10)
    real, intent(in) :: w
    real :: c

    integer :: i, j, k

    k = 3
    !c = 4.2

    do i = 1, 10
        do j = 1, 10
            c = a(i,j) * 2
            b(i,j) = (a(i,j-1)+a(i,j+1))*c
        end do
    end do

end subroutine qux
"""

from psyclone.autodiff.transformations import ADReverseContainerTrans
from psyclone.psyir.frontend.fortran import FortranReader
from psyclone.psyir.backend.fortran import FortranWriter
from psyclone.psyir.nodes import Container, Routine, Reference, Call, OMPParallelDoDirective, Schedule, Loop
from psyclone.autodiff import ADJointReversalSchedule, ADSplitReversalSchedule, ADLinkReversalSchedule

# Front- and backend
freader = FortranReader()
fwriter = FortranWriter()

# PSyIR
psy = freader.psyir_from_source(source)

# Get the container
container = psy.walk(Container)[0]
routine = container.children[0]
loop = routine.walk(Loop)[0]
loop.replace_with(OMPParallelDoDirective(children=[loop.copy()]))

# Initialize the container transformation
container_trans = ADReverseContainerTrans()

# What to differentiate and wrt to what
# d{dependent}/d{independent}
# routine_name = 'foo'
# dependent_vars = ['f','g']
# independent_vars = ['x','w']
routine_name = 'qux'
dependent_vars = ['b']
independent_vars = ['a','w']

# Reversal schedules that are available at this date
split_schedule = ADSplitReversalSchedule()
joint_schedule = ADJointReversalSchedule()
link_schedule = ADLinkReversalSchedule( strong_links=[['foo', 'bar']],
                                        #weak_links=[['foo', 'bar']],
                                        default_link='weak')
reversal_schedule = joint_schedule

# Available options, for now
options = {'verbose': False,                     # default is False
           'jacobian': False,                    # default is False
           'simplify': True,                    # default is True
           'simplify_times': 5,                 # default is 5
           'inline_operation_adjoints': True,   # default is True
           "tape_outside_loops": True,
           "post_process_tbr": True}

# Apply the transformation
result = container_trans.apply(container, 
                               routine_name, 
                               dependent_vars, 
                               independent_vars, 
                               reversal_schedule, 
                               options=options)



# print(result.view())

from psyclone.line_length import FortLineLength

# Transformed source
print("==================================================\n"*3)
print(FortLineLength().process(fwriter(result)))


subroutine qux(a, b, w)
  real, dimension(10,10), intent(inout) :: a
  real, dimension(10,10), intent(out) :: b
  real, intent(in) :: w
  real :: c
  integer :: i
  integer :: j
  integer :: k

  k = 3
  !$omp parallel do default(shared), private(c,i,j)
  do i = 1, 10, 1
    do j = 1, 10, 1
      c = a(i,j) * 2
      b(i,j) = (a(i,j - 1) + a(i,j + 1)) * c
    enddo
  enddo
  !$omp end parallel do

end subroutine qux
subroutine qux_rec(a, b, w, value_tape_integer_qux, value_tape_real_qux, ctrl_tape_boolean_qux)
  real, dimension(10,10), intent(inout) :: a
  real, dimension(10,10), intent(out) :: b
  real, intent(in) :: w
  integer, dimension(3), intent(out) :: value_tape_integer_qux
  real, dimension(201), intent(out) :: value_tape_real_qux
  logical, dimension(0), intent(out) :: ctrl_tape_boolean_qux
  real :: c
  integer :: i
  integer :: j
  integer :: k
  integer :: value_tape_integer_qux_do_offset
  integer :: value_tape_integer_qux_offset
  integer :: value_tape_real_qux_do_offse